<a href="https://colab.research.google.com/github/sharminislamshroddha/machine_learning/blob/main/ensemble_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

## **Loading and Spliting the Dataset**

In [ ]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
np.unique(y)

array([0, 1])

## **Voting Classifiers**

In [ ]:
voting_clf = VotingClassifier(
    estimators=[
        ('LogisticRegression', LogisticRegression(random_state = 42)),
        ('RandomForestClassifier', RandomForestClassifier(random_state=42)),
        ('SVC', SVC(random_state=42))
    ]
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(random_state=42)),
                             ('RandomForestClassifier',
                              RandomForestClassifier(random_state=42)),
                             ('SVC', SVC(random_state=42))])

In [ ]:
print('Individual score: ')

for name, classifier in voting_clf.named_estimators_.items():

  print('{0} = {1}'.format(name, classifier.score(X_test, y_test)))

Individual score: 
LogisticRegression = 0.864
RandomForestClassifier = 0.896
SVC = 0.896


In [ ]:
# view the prediction for 1 data point

[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

In [ ]:
print('Voting classification score {0}'.format(voting_clf.score(X_test, y_test)))

Voting classification score 0.912


## **Bagging and Pasting**

#### **Bagging**

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,
                            max_samples=100,
                            n_jobs=-1,
                            random_state=42
                            )

bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

In [ ]:
y_pred = bag_clf.predict(X_test)

print('Accuracy score: {0}'.format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.904


#### **Pasting**

In [ ]:
pas_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500,
                            max_samples=100,
                            bootstrap = False, # indicate that pasting is used
                            n_jobs=-1,
                            random_state=42
                            )

pas_clf.fit(X_train, y_train)

BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_samples=100, n_estimators=500, n_jobs=-1,
                  random_state=42)

In [ ]:
y_pred = pas_clf.predict(X_test)

print('Accuracy score: {0}'.format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.92


#### **A Random Forests is equicalent to a bag of dicision trees:**

In [ ]:
# Create RandomForestClassifier Model

rf_clf = RandomForestClassifier(n_estimators=500, 
                                max_leaf_nodes=16,
                                n_jobs=-1, 
                                random_state=42
                               )

rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

In [ ]:
# Create BaggingClassifier Model

bag_clf = BaggingClassifier(
                            DecisionTreeClassifier(max_features="sqrt", 
                                                   max_leaf_nodes=16
                                                  ),
                            n_estimators=500, 
                            n_jobs=-1, 
                            random_state=42)

bag_clf.fit(X_train, y_train)

y_pred_bag = bag_clf.predict(X_test)

In [ ]:
print('Are prediction of both classifire same? {0}'.format(np.all(y_pred_bag == y_pred_rf)))

Are prediction of both classifire same? True


## **Boosting**

#### **Adaptive Boosting (AdaBoost)**

In [ ]:
ada_clf = AdaBoostClassifier(
                  DecisionTreeClassifier(max_depth=1),
                  n_estimators=30,
                  learning_rate=0.5,
                  random_state=42
)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

In [ ]:
y_pred = ada_clf.predict(X_test)

print('Accuracy score: {0}'.format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.904


#### **Gradient Boosting**

In [ ]:
gbc = GradientBoostingClassifier(
                max_depth=2,
                n_estimators=500,
                learning_rate=0.05,
                random_state=42
)

gbc.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.05, max_depth=2, n_estimators=500,
                           random_state=42)

In [ ]:
y_pred = gbc.predict(X_test)

print('Accuracy score: {0}'.format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.896


#### **Stacking**

In [ ]:
stacking_clf = StackingClassifier(
    estimators=[
        ('LogisticRegression', LogisticRegression(random_state=42)),
        ('RandomForestClassifier', RandomForestClassifier(random_state=42)),
        ('SVC', SVC(probability=True, random_state=42))
    ],

    final_estimator = RandomForestClassifier(random_state=42),
    cv = 5 # number of cross-validation folds
)

stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('LogisticRegression',
                                LogisticRegression(random_state=42)),
                               ('RandomForestClassifier',
                                RandomForestClassifier(random_state=42)),
                               ('SVC', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=42))

In [ ]:
print('Accuracy Score: {0}'.format(stacking_clf.score(X_test, y_test)))

Accuracy Score: 0.912


#### **XGBoost**

In [ ]:
print('Unique classes in the dataset: {0}\n'.format(np.unique(y)))

Unique classes in the dataset: [0 1]



In [ ]:
xgb_clf = xgb.XGBClassifier(objective = 'binary:logistic', # 'multi:softmax' for multiclass calssification
                            n_estimators=100, 
                            random_state = 42
                           )

xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [ ]:
y_pred = xgb_clf.predict(X_test)

print('Accuracy Score: {0}'.format(accuracy_score(y_test, y_pred)))

Accuracy Score: 0.872
